In [3]:
import json
import csv
import sys
from datetime import datetime
import pandas as pd
from collections import defaultdict

csv.field_size_limit(sys.maxsize)

all_ratings = pd.read_csv('./work/data/collected_animelists.csv')

class RatingSumCounter:
    sum = 0
    count = 0

# Compute total + average ratings per anime
rating_sums_by_anime = defaultdict(lambda: RatingSumCounter())

i = 0
for row in all_ratings.itertuples():
    anime_id = row.anime_id
    rating = row.my_score
    ratings = rating_sums_by_anime[anime_id]
    ratings.sum += rating
    ratings.count += 1
    
    i += 1
    if i % 1000000 == 0:
        print(f"Processed {i} ratings")

# Exported from MySQL
with open('./work/data/anime-metadata.csv') as f:
    reader = csv.reader(f, delimiter=',', quotechar='"')
    # skip header row
    next(reader)
    
    # Read by the webapp backend
    with open('./work/data/processed-metadata.csv', 'wt') as f:
        writer = csv.writer(f, delimiter=',', quotechar='"')
        headers = ['id', 'title', 'title_english', 'related_anime', 'recommendations', 'aired_from_year', 'rating_count', 'average_rating', 'media_type']
        writer.writerow(headers)

        for row in reader:
            anime_id = int(row[0])
            rating_count = rating_sums_by_anime[anime_id].count
            if rating_count < 100:
                continue
            metadata = json.loads(row[1])
            title = metadata['title']
            title_english = metadata['alternative_titles']['en'] if 'en' in metadata['alternative_titles'] else title
            related_anime = json.dumps(metadata.get('related_anime', []))
            recommendations = json.dumps(metadata.get('recommendations', []))
            aired_from_year = int(metadata.get('start_date', '1900')[:4])
            average_rating = rating_sums_by_anime[anime_id].sum / rating_count if rating_count > 0 else 0
            media_type = metadata.get('media_type') or 'unknown'
            if metadata.get('media_type') is None:
                print(f"No media type for {anime_id}")
            writer.writerow([anime_id, title, title_english, related_anime, recommendations, aired_from_year, rating_count, average_rating, media_type])

No media type for 8
No media type for 15
No media type for 17
No media type for 18
No media type for 23
No media type for 24
No media type for 26
No media type for 27
No media type for 28
No media type for 29
No media type for 46
No media type for 49
No media type for 50
No media type for 51
No media type for 52
No media type for 53
No media type for 54
No media type for 55
No media type for 56
No media type for 58
No media type for 59
No media type for 60
No media type for 61
No media type for 62
No media type for 64
No media type for 65
No media type for 67
No media type for 68
No media type for 69
No media type for 71
No media type for 72
No media type for 73
No media type for 74
No media type for 76
No media type for 77
No media type for 80
No media type for 81
No media type for 82
No media type for 83
No media type for 84
No media type for 85
No media type for 86
No media type for 87
No media type for 88
No media type for 89
No media type for 91
No media type for 92
No media type 